In [ ]:
import logging

# Configurations
from configuration.llm_inference_configuration import APILLMConfiguration
from configuration.embedding_inference_configuration import APIEmbeddingModelConfiguration, LocalEmbeddingModelConfiguration
# System components
from llm.azure_openai import AzureOpenAILLM
from embedding.local_embedding import LocalEmbeddingModel
from embedding.request_embedding import RequestEmbeddingModel
# Prompt
from prompt.zero_shot import ZeroShotPrompt
from prompt.few_shot import FewShotPrompt
from prompt.user_message import UserMessagePrompt
from prompt.assistant_message import AssistantMessagePrompt
# Memory
from base_classes.memory.memory_atom import AbstractMemoryAtom
from base_classes.memory.memory_block import AbstractMemoryBlock
from base_classes.memory.memory import AbstractMemory
from base_classes.memory.datatypes.data_item import PromptDataItem

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s\t\t%(levelname)s\t%(message)s',
    handlers=[logging.StreamHandler()]
)


# 1. System component test

## 1.1. LLM test

### 1.1.1. Azure OpenAI test

In [ ]:
gpt_4o_azure_configuration = APILLMConfiguration()
gpt_4o_azure_configuration.load("configuration/yaml/llm/azure_gpt_4o.yaml")
azure_gpt_4o = AzureOpenAILLM(gpt_4o_azure_configuration)

try:
    azure_gpt_4o_1 = AzureOpenAILLM(gpt_4o_azure_configuration)
except Exception as e:
    logging.info("LLM unique ID test successful")
    logging.error(f"Failed to create AzureOpenAILLM instance: {e}")

In [ ]:
sample_zero_shot_message = ZeroShotPrompt(
    [
        {
            "role": "user", 
            "content": "Health check. Say 'hi' to start the conversation."
        }
    ]
)

sample_zero_shot_message_responses = azure_gpt_4o.query(query=sample_zero_shot_message.prompt, num_responses=2)
print(azure_gpt_4o.get_response_texts(sample_zero_shot_message_responses))

In [ ]:
sample_few_shots_message = FewShotPrompt(
    [
        {
            "role": "system", 
            "content": "You are NetMind assistant. You task is to answer to the user anything about Viettel Group."
        },
        {
            "role": "user", 
            "content": "Tell me about Viettel Group."
        },
        {
            "role": "assistant", 
            "content": "Viettel Group is a Vietnamese multinational telecommunications company headquartered in Hanoi, Vietnam. It is a state-owned enterprise and operated by the Ministry of Defence. You can find out more about Viettel Group at https://viettel.vn/."
        },
        {
            "role": "user",
            "content": "What is the revenue of Viettel Group?"
        }
    ]
)

sample_few_shots_message_responses = azure_gpt_4o.query(query=sample_few_shots_message.prompt, num_responses=2)
print(azure_gpt_4o.get_response_texts(sample_few_shots_message_responses))

## 1.2. Embedding test

### 1.2.1. Request embedding model test

In [ ]:
all_mini_v2_configuration = APIEmbeddingModelConfiguration()
all_mini_v2_configuration.load("configuration/yaml/embedding/all-MiniLM-L6-v2.yaml")
all_mini_v2 = RequestEmbeddingModel(all_mini_v2_configuration)

In [ ]:
all_mini_v2.encode("Hello world")

In [ ]:
all_mini_v2 = RequestEmbeddingModel(all_mini_v2)

In [ ]:
a = {"b": 1, "c": 2}
list(a.keys())

## 1.3. Tool test

### 1.3.1. Python code runner tool

In [ ]:
from tools.python_code_runner import PythonCodeRunnerTool
from configuration.tool_configuration import ToolConfiguration

In [ ]:
python_code_runner_config = ToolConfiguration()
python_code_runner_config.load("configuration/yaml/tools/python_code_runner.yaml")
python_code_runner = PythonCodeRunnerTool(python_code_runner_config)

In [ ]:
input_code_list = [
    {
        "code_string": "a = 11\nb = 20\nresult = a + b\n\ndef greet(name):\n return f'Hello, {name}!'\n \nmessage = greet('World')",
        "id": "1"
    },
    {
        "code_string": "def multiply(x, y):\n\treturn x * y\n\nnum1 = 5\nnum2 = 3\nresult = multiply(num1, num2)\n\nprint(f'The product of {num1} and {num2} is {result}.')",
        "id": "2"
    }
]

results = python_code_runner.execute(input_code_list=input_code_list)
results

# 2. Memory test

In [ ]:
prompt_data_1 = [{
    'role': 'user',
    'content': 'Calculate the sum of 11 and 20.'
}]
mem_atom_1 = AbstractMemoryAtom(data=PromptDataItem(UserMessagePrompt(prompt_data_1)))

prompt_data_2 = [{
    'role': 'assistant',
    'content': 'In order to calculate the sum of two numbers, we need to do the following:\n\t1. Write a Python code with add_sum() function, receiving 2 variables.\n\t2. Execute the code with the given variables.\n\t3. Return the result of the sum.'
}]
mem_atom_2 = AbstractMemoryAtom(data=PromptDataItem(AssistantMessagePrompt(prompt_data_2)))

prompt_data_3 = [{
    'role': 'assistant',
    'content': 'Here is the Python code to calculate the sum of two numbers:\n\na = 11\nb = 20\nresult = a + b\n\nprint(f"The sum of {a} and {b} is {result}.")'
}]
mem_atom_3 = AbstractMemoryAtom(data=PromptDataItem(AssistantMessagePrompt(prompt_data_3)))

prompt_data_4 = [{
    'role': 'assistant',
    'content': 'After executing the code, we got the sum of 11 and 20 is 31.'
}]
mem_atom_4 = AbstractMemoryAtom(data=PromptDataItem(AssistantMessagePrompt(prompt_data_4)))

In [ ]:
AbstractMemoryAtom.get_mematom_ids()

In [ ]:
mem_block = AbstractMemoryBlock()
mem_block.add_memory_atom(mem_atom_1)
mem_block.add_memory_atom(mem_atom_2)
mem_block.add_memory_atom(mem_atom_3)
mem_block.add_memory_atom(mem_atom_4)

mem_block.mem_atom_graph = {
        mem_atom_1.mem_atom_id: [mem_atom_2.mem_atom_id],
        mem_atom_2.mem_atom_id: [mem_atom_3.mem_atom_id, mem_atom_4.mem_atom_id],
        mem_atom_3.mem_atom_id: [],
        mem_atom_4.mem_atom_id: []
    }

In [ ]:
memory = AbstractMemory()
memory.add_memory_block(mem_block)

In [ ]:
memory.get_memory_block_by_id(mem_block.mem_block_id)

In [ ]:
from annotated_docs.json_schema import as_json_schema

class A:
    """
    This is a class A.
    """
    
    def __init__(self, a: int, b: str):
        """
        This is the constructor of class A.
        """
        self.a = a
        self.b = b
    @classmethod
    def get_a(self, m: int) -> int:
        """
        This is the get_a method of class A.
        """
        return self.a
    
    def get_b(self) -> str:
        """
        This is the get_b method of class A.
        """
        return self.b

a = A(a = 2, b = "a")

as_json_schema(A.get_a)
# a = A(a = 2, b = "a")
# as_json_schema(A.get_a())

In [ ]:
import logging
from abc import ABC

class CustomLogger(logging.Logger):
    def __init__(self, name, containing_class_name, level=logging.NOTSET):
        super().__init__(name, level)
        # Explicitly set logger level to DEBUG to ensure all messages are processed
        self.setLevel(logging.DEBUG)
        self.containing_class_name = containing_class_name
        self.level_formats = {
            logging.DEBUG: {"prefix": f"[🔧 {containing_class_name}] ", "suffix": ""},
            logging.INFO: {"prefix": f"[ℹ️ {containing_class_name}] ", "suffix": ""},
            logging.WARNING: {"prefix": f"[⚠️ {containing_class_name}] ", "suffix": ""},
            logging.ERROR: {"prefix": f"[❌ {containing_class_name}] ", "suffix": ""},
            logging.CRITICAL: {"prefix": f"[‼️ {containing_class_name}] ", "suffix": ""}
        }

    def _log(self, level, msg, args, exc_info=None, extra=None, stack_info=False, stacklevel=1):
        fmt = self.level_formats.get(level, {"prefix": f"[📜 {self.containing_class_name}] ", "suffix": ""})
        modified_msg = f"{fmt['prefix']}{msg}{fmt['suffix']}"
        super()._log(level, modified_msg, args, exc_info, extra, stack_info, stacklevel)

    def debug(self, msg, *args, **kwargs):
        self._log(logging.DEBUG, msg, args, **kwargs)

    def info(self, msg, *args, **kwargs):
        self._log(logging.INFO, msg, args, **kwargs)

    def warning(self, msg, *args, **kwargs):
        self._log(logging.WARNING, msg, args, **kwargs)

    def error(self, msg, *args, **kwargs):
        self._log(logging.ERROR, msg, args, **kwargs)

    def critical(self, msg, *args, **kwargs):
        self._log(logging.CRITICAL, msg, args, **kwargs)

class HasLoggerClass(ABC):
    logger: CustomLogger

    def __init__(self):
        # Initialize logger with the class name of the containing class
        self.logger = CustomLogger(self.__class__.__name__, self.__class__.__name__)

# Example implementation
class MyClass(HasLoggerClass):
    def do_something(self):
        self.logger.warning("Warning about something")
        self.logger.error("Error in something")
        self.logger.critical("Critical issue in something")
        self.logger.debug("Debugging something")
        self.logger.info("Info about something")

# Example usage
if __name__ == "__main__":
    # Set the custom logger class
    logging.setLoggerClass(CustomLogger)
    
    # Configure logging with basicConfig
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s\t\t%(levelname)s\t%(message)s',
        handlers=[logging.StreamHandler()]
    )
    
    # Create an instance of MyClass
    obj = MyClass()
    
    # Ensure the logger propagates to the root logger to use basicConfig settings
    obj.logger.propagate = True
    
    # Test the logger
    obj.do_something()